In [ ]:
import os 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
import keras
from glob import glob

from keras.layers import add,Input,Layer,Conv2D,multiply,Dropout,MaxPool2D,Concatenate,Conv2DTranspose,BatchNormalization
from keras.callbacks import Callback, ModelCheckpoint
from keras.models import Model

In [ ]:
path ='/kaggle/input/semantic-segmentation' 
raw_images_paths=[os.path.join(path, image) for image in os.listdir(path) if image.endswith(".jpg")]
label_images_paths = [image.replace(".jpg",".jpg___fuse.png") for image in raw_images_paths]

In [ ]:
def load_image(path,size=256):
    # load,resize,and normalize images
    image=load_img(path)
    image=img_to_array(image)
    image=cv2.resize(image,(size,size))
    image = tf.cast(image, tf.float32)
    image = image/255.
    return image

In [ ]:
def load_data(raw_images_paths,label_images_paths,size=256):
    #create zero np array to save the images to it later 
    images,label_maps=np.zeros(shape=(len(raw_images_paths),size,size,3)),np.zeros(shape=(len(label_images_paths),size,size,3))
    for i ,(image_path,label_path)in tqdm(enumerate(zip(raw_images_paths,label_images_paths))):
        image,label_map=load_image(image_path,size=size),load_image(label_path,size=size)
        images[i],label_maps[i]=image,label_map
    return images,label_maps

In [ ]:
images, label_maps = load_data(raw_images_paths, label_images_paths)


In [ ]:
def show_image(image, title=None):
    plt.imshow(image)
    plt.title(title)
    plt.axis('off')


In [ ]:
from keras.callbacks import Callback, ModelCheckpoint
class ShowProgress(Callback):
    def on_epoch_end(self, epoch, logs=None):
        id = np.random.randint(len(images))
        image = images[id]
        mask = label_maps[id]
        pred_mask = self.model(tf.expand_dims(image,axis=0))[0]
        
        plt.figure(figsize=(10,8))
        plt.subplot(1,3,1)
        show_image(image, title="Original Image")
        
        plt.subplot(1,3,2)
        show_image(mask, title="Original Mask")
        
        plt.subplot(1,3,3)
        show_image(pred_mask, title="Predicted Mask")
            
        plt.tight_layout()
        plt.show()

In [ ]:
show_progress = [ModelCheckpoint("UNet-Footbal-Player-Segmentation.h5", save_best_only=True),ShowProgress()]

In [ ]:
def encoder_block(X, filters, rate=0.2, pooling=True):
    bn = BatchNormalization()
    c1 = Conv2D(filters, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')
    drop = Dropout(rate)
    c2 = Conv2D(filters, kernel_size=3, strides=1, padding='same', activation='relu', kernel_initializer='he_normal')
    pool = MaxPool2D()

    x = c2(drop(c1(bn(X))))
    
    if pooling:
        y = pool(x)
        return x, y
    return x

In [ ]:
def decoder_block(x, skip_x, filters, rate):
    cT = Conv2DTranspose(filters, kernel_size=3, strides=2, padding='same', activation='relu', kernel_initializer='he_normal')
    bn = BatchNormalization()
    skip = Concatenate()

    y = cT(bn(x))
    y = encoder_block(skip([y, skip_x]), filters, rate, pooling=False)

    return y

In [ ]:
InputL = Input(shape=(256, 256, 3), name="InputImage")

# Encoder Block 
c1, p1 = encoder_block(InputL, filters=64, rate=0.1)
c2, p2 = encoder_block(p1, filters=128, rate=0.1)
c3, p3 = encoder_block(p2, filters=256, rate=0.2)
c4, p4 = encoder_block(p3, filters=512, rate=0.2)

# Encoding Layer
encodings = encoder_block(p4, filters=512, rate=0.3, pooling=False)

# Decoder Block
d = decoder_block(encodings, c4, filters=512, rate=0.2)
d = decoder_block(d, c3, filters=256, rate=0.2)
d = decoder_block(d, c2, filters=128, rate=0.1)
d = decoder_block(d, c1, filters=64, rate=0.1)

# Output
conv_out = Conv2D(3, kernel_size=3, padding='same', activation='sigmoid', name="Segmentator")(d)

# Model
model = Model(InputL, conv_out, name="UNet")
model.summary()

# Compile Model
model.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:

model.fit(
    images, label_maps,
    validation_split=0.1,
    epochs=100,
    batch_size=16,
    steps_per_epoch=6,
    callbacks=show_progress

)